<a href="https://colab.research.google.com/github/Ayanlola2002/Stutern/blob/master/Stutern_Group_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
all_accounts=pd.read_csv('/content/drive/My Drive/raw/all_accounts.csv')
all_balances=pd.read_csv('/content/drive/My Drive/raw/all_balances.csv')
all_demographics=pd.read_csv('/content/drive/My Drive/raw/all_demographics.csv',encoding='latin',sep='|')
all_investments=pd.read_csv('/content/drive/My Drive/raw/all_investments.csv')
all_loans=pd.read_csv('/content/drive/My Drive/raw/all_loans.csv')
all_loans_application=pd.read_csv('/content/drive/My Drive/raw/all_loans_application.csv')
all_transactions_digital=pd.read_csv('/content/drive/My Drive/raw/all_transactions_digital.csv')
bureau_score=pd.read_csv('/content/drive/My Drive/raw/bureau_score.csv')
all_dictionary=pd.read_csv('/content/drive/My Drive/raw/dictionary.csv',skiprows=[0])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,19,26,28,30,35,36,37,38,39,61,71,73,74,85,100,103,114,115,125,126) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#we use the all_loans_applications customer id 
#this indicates all those customers that apply for loans 
#it may also be possible that a customer ask for multiple loans
print(all_loans.shape)
print(all_loans_application.shape)

(278505, 10)
(23172, 8)


In [0]:
#setting customerId as index 
all_loans_application.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [0]:
#extracting customer ID from  all_loan_applications
important_indexes=all_loans_application.index.unique()

In [0]:
#resetting of all_accounts index
all_accounts.set_index('CUSTOMER_UNIQUE_ID',inplace=True)


In [7]:
all_accounts=all_accounts.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [8]:
all_accounts['RECORD_STAT'].value_counts()

O    27281
C     1776
Name: RECORD_STAT, dtype: int64

In [9]:
all_accounts.shape

(29068, 4)

In [0]:
all_accounts.drop(columns='MASKED_ACCOUNT',inplace=True)

In [11]:
#checking different type of currency
all_accounts['CCY'].value_counts()

NGN    26765
USD     2082
GBP      146
EUR       64
Name: CCY, dtype: int64

In [0]:
#function to convert different currency to naira
def con(data):
    data.loc[data.CCY=='USD','ACY_AVL_BAL']=data.loc[data.CCY=='USD','ACY_AVL_BAL'].apply(lambda x: x*365)
    data.loc[data.CCY=='EUR','ACY_AVL_BAL']=data.loc[data.CCY=='EUR','ACY_AVL_BAL'].apply(lambda x: x*398)
    data.loc[data.CCY=='GBP','ACY_AVL_BAL']=data.loc[data.CCY=='GPB','ACY_AVL_BAL'].apply(lambda x: x*467)
    data.loc[data.CCY=='JPY','ACY_AVL_BAL']=data.loc[data.CCY=='JPY','ACY_AVL_BAL'].apply(lambda x: x*3.31)
    return data

In [0]:
#changing all account to naira notes
all_accounts=con(all_accounts)

In [0]:
#convert index to int
all_accounts.index=all_accounts.index.astype('int')

In [0]:
#sort index
all_accounts.sort_index(inplace=True)

In [16]:
#meaning of ACY_AVL_BAL
all_dictionary[all_dictionary['FEATURES']=='ACY_AVL_BAL']

,FEATURES,DESCRIPTION,DATA TYPE,Unnamed: 3,Unnamed: 4
14,ACY_AVL_BAL,Specifies the available balance in the account...,NUMBER,NaN,NaN


In [17]:
#some of the ACY_AVL_BAL IS negative we will assumed it is zero 
all_accounts['ACY_AVL_BAL'].value_counts()

 0.00          12405
-52.50           243
-105.00           69
 950.00           33
 1.00             26
               ...  
 77773.41          1
 4058776.43        1
 125.53            1
 14164.08          1
-145.34            1
Name: ACY_AVL_BAL, Length: 14389, dtype: int64

In [0]:
#for all ACY_AVL_BAL that is negative RECORD STAT IS C we convert to zero so as not to affect our model
all_accounts.loc[(all_accounts.ACY_AVL_BAL!=0) & (all_accounts.RECORD_STAT=="C"),'ACY_AVL_BAL']=0

In [19]:
all_accounts.index.value_counts()

69090004     34
578029002    27
172877006    25
657601000    23
679735005    22
             ..
952341006     1
73406011      1
234473009     1
806118002     1
909990007     1
Name: CUSTOMER_UNIQUE_ID, Length: 17986, dtype: int64

In [20]:
#most of the customer id appears more than ones so we can group by like the example below
all_accounts.loc[69090004]

,RECORD_STAT,ACY_AVL_BAL,CCY
CUSTOMER_UNIQUE_ID,,,
69090004,O,0.00,USD
69090004,C,0.00,NGN
69090004,O,37128.66,NGN
69090004,O,0.00,NGN
69090004,O,28.09,NGN
69090004,O,0.00,NGN
69090004,O,0.00,NGN
69090004,O,1262799.23,NGN
69090004,O,89483.40,USD


In [0]:
#currency was removed because all ACY_AVL_BAL was coverted to same currency naira
all_accounts_data=all_accounts.groupby([all_accounts.index])[['ACY_AVL_BAL']].sum()

In [22]:
all_accounts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29068 entries, 56000 to 999944001
Data columns (total 3 columns):
RECORD_STAT    29057 non-null object
ACY_AVL_BAL    28938 non-null float64
CCY            29057 non-null object
dtypes: float64(1), object(2)
memory usage: 1.5+ MB


In [23]:
#there will be no need to merge CCY column to all_accounts dataframe since we converted all currency to naira.
all_accounts_data.head()

,ACY_AVL_BAL
CUSTOMER_UNIQUE_ID,
56000,136043.48
62005,1541719.95
104008,12364.06
175007,20029.55
200006,61489.56


In [0]:
all_balances.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [25]:
#focussing on only those who made loan applications according to all_loan_applications
all_balances=all_balances.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [26]:
#checking for null values
all_balances.isnull().sum()

MASKED_ACCOUNT    490
VAL_DT            490
BAL               490
dtype: int64

In [0]:
#trying to select the last date balance of customers who applied for loans
#merger=all_balances.groupby([all_balances.index,all_balances.MASKED_ACCOUNT])[['VAL_DT']].max()

In [0]:
all_balances_data=all_balances.sort_values('VAL_DT').groupby(all_balances.index).tail(1)

In [31]:
all_balances_data.head()

,MASKED_ACCOUNT,VAL_DT,BAL
CUSTOMER_UNIQUE_ID,,,
41981006,00******68,10/1/2019,40821.02
737790008,06******08,10/1/2019,323.75
521640001,00******56,10/1/2019,11974.56
391254009,00******30,10/1/2019,6813.05
657601000,00******15,10/1/2019,14998.88


In [34]:
all_balances_data.isnull().any(axis=1).sum()

4

In [37]:
#drop rows with na values
all_balances_data.dropna()

,MASKED_ACCOUNT,VAL_DT,BAL
CUSTOMER_UNIQUE_ID,,,
41981006,00******68,10/1/2019,40821.02
737790008,06******08,10/1/2019,323.75
521640001,00******56,10/1/2019,11974.56
391254009,00******30,10/1/2019,6813.05
657601000,00******15,10/1/2019,14998.88
...,...,...,...
801451003,07******68,9/9/2019,85273.88
588148009,00******46,9/9/2019,1097.83
13231006,00******65,9/9/2019,40002.64


In [39]:
all_balances_data.isna().sum()

MASKED_ACCOUNT    4
VAL_DT            4
BAL               4
dtype: int64